In [1]:
import numpy as np
import librosa
import random
from tensorflow.keras import utils

In [2]:
npzfile = np.load('data/all_targets_sets.npz')
print(npzfile.files)

X_data = npzfile['arr_0']
y_data = npzfile['arr_1']
print(X_data.shape, y_data.shape)

['arr_0', 'arr_1']
(999, 1280, 128) (999,)


In [3]:
X_data_repeat = []
for x in X_data:
    X_data_repeat.extend(np.split(x, 10))

X_data_repeat = np.array(X_data_repeat)
y_data_repeat = np.repeat(y_data, 10, axis=0)
print(X_data_repeat.shape, y_data_repeat.shape)

(9990, 128, 128) (9990,)


In [4]:
X_data = X_data_repeat
y_data = y_data_repeat

In [5]:
y_data = utils.to_categorical(y_data, num_classes=10).astype(int)
print(X_data.shape, y_data.shape)

(9990, 128, 128) (9990, 10)


In [6]:
training_data = []
for i in range(X_data.shape[0]):
    training_data.append((X_data[i], y_data[i]))

In [7]:
def prepare_data(data):
    X = []
    y = []

    for frames, labels in data:
        X.append(frames)
        y.append(labels)
    return X, y

In [8]:
def partition_data():
    # Randomize training set
    random.shuffle(training_data)

    X_train, y_train = prepare_data(training_data)

    # Calculate validation and test set sizes
    val_set_size = int(len(training_data) * 0.1)
    test_set_size = int(len(training_data) * 0.1)

    # Break x apart into train, validation, and test sets
    X_valid = X_train[:val_set_size]
    X_test = X_train[val_set_size:(val_set_size + test_set_size)]
    X_train = X_train[(val_set_size + test_set_size):]

    # Break y apart into train, validation, and test sets
    y_valid = y_train[:val_set_size]
    y_test = y_train[val_set_size:(val_set_size + test_set_size)]
    y_train = y_train[(val_set_size + test_set_size):]

    print("Train set size: " + str(len(X_train)))
    print("Validation set size: " + str(len(X_valid)))
    print("Test set size: " + str(len(X_test)))
    
    return np.array(X_train), np.array(y_train), np.array(X_valid), np.array(y_valid), np.array(X_test), np.array(y_test)

In [9]:
X_train, y_train, X_valid, y_valid, X_test, y_test = partition_data()

Train set size: 7992
Validation set size: 999
Test set size: 999


In [10]:
### Convert the scale of training data
X_train_raw = librosa.core.db_to_power(X_train, ref=1.0)
X_train_log = np.log(X_train_raw)
print(np.amin(X_train_raw), np.amax(X_train_raw), np.mean(X_train_raw))
print(np.amin(X_train_log), np.amax(X_train_log), np.mean(X_train_log))

1e-08 1.0000008783668917 0.0036015789420766535
-18.420680743952367 8.783665059016772e-07 -9.847497308221024


In [11]:
X_valid_raw = librosa.core.db_to_power(X_valid, ref=1.0)
X_valid_log = np.log(X_valid_raw)
print(np.amin(X_valid_raw), np.amax(X_valid_raw), np.mean(X_valid_raw))
print(np.amin(X_valid_log), np.amax(X_valid_log), np.mean(X_valid_log))

1e-08 1.0000008783668917 0.003574649189395486
-18.420680743952367 8.783665059016772e-07 -9.86975440761025


In [12]:
X_test_raw = librosa.core.db_to_power(X_test, ref=1.0)
X_test_log = np.log(X_test_raw)
print(np.amin(X_test_raw), np.amax(X_test_raw), np.mean(X_test_raw))
print(np.amin(X_test_log), np.amax(X_test_log), np.mean(X_test_log))

1e-08 1.0000008783668917 0.0035899243579161576
-18.420680743952367 8.783665059016772e-07 -9.892712904340819


In [13]:
X_train, y_train = X_train_log, y_train
X_valid, y_valid = X_valid_log, y_valid
X_test, y_test = X_test_log, y_test

In [14]:
print("Train set shapes: ", X_train.shape, y_train.shape)
print("Validation set shapes: ", X_valid.shape, y_valid.shape)
print("Test set shapes: ", X_test.shape, y_test.shape)

Train set shapes:  (7992, 128, 128) (7992, 10)
Validation set shapes:  (999, 128, 128) (999, 10)
Test set shapes:  (999, 128, 128) (999, 10)


In [15]:
np.savez('data/all_targets_sets_train', X_train, y_train)
np.savez('data/all_targets_sets_valid', X_valid, y_valid)
np.savez('data/all_targets_sets_test', X_test, y_test)